In [1]:
!pip install dmba
import dmba
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pylab as plt
from dmba import classificationSummary

no display found. Using non-interactive Agg backend


In [2]:
df = pd.read_csv('SystemAdministrators.csv')

In [3]:
df.head()

,Experience,Training,Completed task
0,10.9,4,Yes
1,9.9,4,Yes
2,10.4,6,Yes
3,13.7,6,Yes
4,9.4,8,Yes


In [4]:
df['Completed task'] = df['Completed task'].replace(['Yes','No'],[1,0])

In [5]:
df.head()

,Experience,Training,Completed task
0,10.9,4,1
1,9.9,4,1
2,10.4,6,1
3,13.7,6,1
4,9.4,8,1


In [6]:
lda = LinearDiscriminantAnalysis()

In [7]:
lda.fit(df[['Experience','Training']],df['Completed task'])

LinearDiscriminantAnalysis()

In [8]:
classificationSummary(df['Completed task'], lda.predict(df[['Experience','Training']]))

Confusion Matrix (Accuracy 0.9067)

       Prediction
Actual  0  1
     0 58  2
     1  5 10


In [9]:
# Q1: Out of those that completed the tasks, 33% were predicted to not complete the task.

In [10]:
print('Coefficients', lda.coef_)
print('Intercept', lda.intercept_)

Coefficients [[1.44469947 0.14427824]]
Intercept [-13.60512707]


In [16]:
test_record = pd.DataFrame({'Experience':[4],'Training':[6]})

In [18]:
test_df = pd.DataFrame()

In [21]:
test_df['Dec. Function'] = lda.decision_function(test_record)
test_df['Prediction'] = lda.predict(test_record)
test_df['p(Completed)'] = lda.predict_proba(test_record)[:,1]
test_df

,Dec. Function,Prediction,p(Completed)
0,-6.96066,0,0.000948


In [22]:
# Q2 The administrator is classified as 'task not completed'.

In [25]:
predictors = ['Experience','Training']
outcome = 'Completed task'

In [26]:
# partition data
X = df[predictors]
y = df[outcome]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.4, random_state = 1)

In [27]:
# standardize the data
scaler.fit(X_train[['Experience','Training']])

MinMaxScaler()

In [28]:
norm_data = pd.concat([pd.DataFrame(scaler.transform(df[['Experience','Training']]),
                                  columns = ['zExperience','zTraining']),
                     df[['Completed task']]], axis = 1)

In [29]:
X_trainNorm = norm_data.iloc[X_train.index]
X_validNorm = norm_data.iloc[X_valid.index]

In [30]:
X_train = X_trainNorm[['zExperience','zTraining']]
X_valid = X_validNorm[['zExperience','zTraining']]

In [31]:
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [32]:
classificationSummary(y_train, lda.predict(X_train))

Confusion Matrix (Accuracy 0.9111)

       Prediction
Actual  0  1
     0 31  1
     1  3 10


In [33]:
classificationSummary(y_valid, lda.predict(X_valid))

Confusion Matrix (Accuracy 0.9000)

       Prediction
Actual  0  1
     0 25  3
     1  0  2


In [34]:
mlpc = MLPClassifier()

In [35]:
mlpc.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPClassifier()

In [36]:
classificationSummary(y_train, mlpc.predict(X_train))

Confusion Matrix (Accuracy 0.8889)

       Prediction
Actual  0  1
     0 32  0
     1  5  8


In [37]:
classificationSummary(y_valid, mlpc.predict(X_valid))

Confusion Matrix (Accuracy 0.9333)

       Prediction
Actual  0  1
     0 27  1
     1  1  1


In [38]:
# Q3: For the Discriminant Analysis method, the model seemed to perform well on the validation data. However, due to the small size of the dataset it's difficult to say for sure that the model will have high accuracy on future data.
# For the Neural Net method, the model actually performed better on the validation set than the training set. However, the model predicted more false negatives than the Discriminant Analysis method.
# Again, it's hard to evaluate the models' performance on a small dataset, but it seems that the Discriminant Analysis method produces fewer false negatives compared to the Neural Net method